In [1]:
!pip install tensorflow==2.15.0

In [2]:
!pip install keras

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
#import cv2 as cv
import warnings
from keras.callbacks import Callback, ModelCheckpoint
from tensorflow import keras

2024-05-23 12:41:57.338153: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-23 12:41:57.357336: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 12:41:57.357350: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 12:41:57.358218: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-23 12:41:57.362144: I tensorflow/core/platform/cpu_feature_guar

In [4]:
from keras.preprocessing.image import ImageDataGenerator

In [5]:
from sklearn.metrics import classification_report, confusion_matrix

In [6]:
print(tf.__version__)

2.15.0


In [7]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


2024-05-23 12:42:06.962393: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-23 12:42:06.962566: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-23 12:42:06.988301: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
# Extract dataset .zip (Run only once)
!unzip -o -q Dataset.zip  -d Dataset

In [8]:
from keras import backend as K
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, Dense
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.utils import get_source_inputs
from keras.utils import get_file
from tensorflow.python.keras.utils import layer_utils

In [9]:
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

WEIGHTS_PATH = "https://github.com/rcmalli/keras-squeezenet/releases/download/v1.0/squeezenet_weights_tf_dim_ordering_tf_kernels.h5"
WEIGHTS_PATH_NO_TOP = "https://github.com/rcmalli/keras-squeezenet/releases/download/v1.0/squeezenet_weights_tf_dim_ordering_tf_kernels_notop.h5"

# Modular function for Fire Node

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x


# Original SqueezeNet from paper.

def SqueezeNet(include_top=True, weights='imagenet',
               input_tensor=None, input_shape=None,
               pooling=None,
               classes=1000):
    """Instantiates the SqueezeNet architecture.
    """
        
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')




    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)



    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    if include_top:

    
        x = Dropout(0.5, name='drop9')(x)

        x = Convolution2D(100, (1, 1), padding='valid', name='conv10')(x)
        x = Activation('relu', name='relu_conv10')(x)
        x = GlobalAveragePooling2D()(x)
        x = Dense(7,activation = 'softmax', name='loss')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling=='max':
            x = GlobalMaxPooling2D()(x)
        elif pooling==None:
            pass
        else:
            raise ValueError("Unknown argument for 'pooling'=" + pooling)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    model = Model(inputs, x, name='squeezenet')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('squeezenet_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('squeezenet_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')
            
        model.load_weights(weights_path)

    return model

In [10]:
from keras.preprocessing import image

model = SqueezeNet(include_top=True, weights=None,
               input_tensor=None, input_shape=(224,224,3),
               classes=7)

2024-05-23 12:42:14.118661: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-23 12:42:14.118784: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-23 12:42:14.118866: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [11]:
model.summary()

Model: "squeezenet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1 (Conv2D)              (None, 111, 111, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 relu_conv1 (Activation)     (None, 111, 111, 64)         0         ['conv1[0][0]']               
                                                                                                  
 pool1 (MaxPooling2D)        (None, 55, 55, 64)           0         ['relu_conv1[0][0]']          
                                                                                         

In [12]:
def mob_pre_data(i_tensor):
    return (i_tensor / 255).astype(np.float32)

PATH = "Dataset_compressed"
print("path: ", PATH)
BATCH_SIZE = 64
IMG_SIZE = (224, 224)

train_datagen = ImageDataGenerator(rescale = 1./ 255,
                                   validation_split=0.20)

train_ds = train_datagen.flow_from_directory(PATH,
                                             target_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical",
                                             color_mode="rgb",
                                             shuffle=True,
                                             subset='training',
                                             seed=42,
                                             classes=['Clear', 'SlantingHeavyRain', 'VerticalHeavyRain',
                                                      'SlantingMediumRain', 'VerticalMediumRain', 'SlantingLowRain',
                                                      'VerticalLowRain'])

val_ds = train_datagen.flow_from_directory(PATH,
                                           target_size=IMG_SIZE,
                                           batch_size=BATCH_SIZE,
                                           class_mode="categorical",
                                           color_mode="rgb",
                                           shuffle=True,
                                           seed=42,
                                           subset="validation",
                                           classes=['Clear', 'SlantingHeavyRain', 'VerticalHeavyRain',
                                                    'SlantingMediumRain', 'VerticalMediumRain', 'SlantingLowRain',
                                                    'VerticalLowRain'])

print(val_ds.class_indices)

path:  Dataset_compressed
Found 267731 images belonging to 7 classes.
Found 66930 images belonging to 7 classes.
{'Clear': 0, 'SlantingHeavyRain': 1, 'VerticalHeavyRain': 2, 'SlantingMediumRain': 3, 'VerticalMediumRain': 4, 'SlantingLowRain': 5, 'VerticalLowRain': 6}


In [13]:
starter_learning_rate = 0.1
end_learning_rate = 0.001
decay_steps = 10000
learning_rate_fn = keras.optimizers.schedules.PolynomialDecay(
    starter_learning_rate,
    decay_steps,
    end_learning_rate,
    power=0.5)

In [17]:
!export PATH="${PATH}:/usr/local/nvidia/bin:/usr/local/cuda/bin"

In [21]:
base_learning_rate = 0.0001

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_fn),
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [22]:
class EarlyStoppingByAccuracy(Callback):
    def __init__(self, monitor='val_accuracy', value=0.995, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True


callbacks_1 = [
    EarlyStoppingByAccuracy(monitor='val_accuracy', value=0.995, verbose=1),
]

In [ ]:
from os import listdir, remove
from PIL import Image

path = "Dataset_compressed/VerticalLowRain/"
for filename in listdir(path):
    if filename.endswith('.jpg'):
        try:
            img = Image.open(path+filename) # open the image file
            img.verify() # verify that it is, in fact an image
        except (IOError) as e:
            print('Bad file:', filename)
            #print('Removing file ...')
            #remove(path+filename)

In [ ]:
!nvidia-smi

In [23]:
print("Start training ...")
history = model.fit(train_ds,
                    initial_epoch=0,
                    epochs=100,
                    validation_data=val_ds,
                    callbacks=callbacks_1)
print(history)

Start training ...
Epoch 1/100


2024-05-23 12:45:38.457591: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.70MiB (rounded to 5971968)requested by op squeezenet/fire5/squeeze1x1/Conv2D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-05-23 12:45:38.457633: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-05-23 12:45:38.457644: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 165, Chunks in use: 164. 41.2KiB allocated for chunks. 41.0KiB in use in bin. 13.1KiB client-requested in use in bin.
2024-05-23 12:45:38.457650: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 45, Chunks in use: 45. 27.2KiB allocated for chunks. 27.2KiB in use in bin. 26.7KiB client-requested in use in 

ResourceExhaustedError: Graph execution error:

Detected at node squeezenet/fire5/squeeze1x1/Conv2D defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/admin/Environments/jupyter/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/admin/Environments/jupyter/lib/python3.10/site-packages/traitlets/config/application.py", line 1077, in launch_instance

  File "/home/admin/Environments/jupyter/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/admin/Environments/jupyter/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/admin/

096 totalling 52.0KiB
2024-05-23 12:45:38.458804: I external/local_tsl/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 4352 totalling 4.2KiB
2024-05-23 12:45:38.458807: I external/local_tsl/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 4608 totalling 4.5KiB
2024-05-23 12:45:38.458810: I external/local_tsl/tsl/framework/bfc_allocator.cc:1103] 5 Chunks of size 6912 totalling 33.8KiB
2024-05-23 12:45:38.458814: I external/local_tsl/tsl/framework/bfc_allocator.cc:1103] 4 Chunks of size 8192 totalling 32.0KiB
2024-05-23 12:45:38.458817: I external/local_tsl/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 9472 totalling 9.2KiB
2024-05-23 12:45:38.458820: I external/local_tsl/tsl/framework/bfc_allocator.cc:1103] 14 Chunks of size 16384 totalling 224.0KiB
2024-05-23 12:45:38.458823: I external/local_tsl/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 28416 totalling 27.8KiB
2024-05-23 12:45:38.458826: I external/local_tsl/tsl/framework/bfc_allocator.cc:1103] 4 Chunks 

In [ ]:
print("Plotting history ...")

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()), 1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0, 1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

print("Model test ...")
loss, accuracy = model.evaluate(val_ds)
print('Test accuracy :', accuracy)

class_names = ['Clear', 'SlantingHeavyRain', 'VerticalHeavyRain',
               'SlantingMediumRain', 'VerticalMediumRain', 'SlantingLowRain',
               'VerticalLowRain']

In [ ]:
# Retrieve a batch of images from the test set
image_batch, label_batch = next(val_ds)
predictions = model.predict(image_batch)
predictions = np.argmax(predictions, 1)

# Apply a sigmoid since our model returns logits

predictions = tf.where(predictions < 0.5, 0, 1)
label_batch = np.argmax(label_batch, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(8):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i])
    plt.title(class_names[predictions[i]])
    plt.axis("off")

predictions = np.array([])
labels = np.array([])
for im in range(len(val_ds)):
    x, y = val_ds[im]
    predictions = np.concatenate([predictions, np.argmax(model.predict(x), axis=1)])
    labels = np.concatenate([labels, np.argmax(y, axis=1)])

tf.math.confusion_matrix(labels=labels, predictions=predictions).numpy()

import seaborn as sns

confusion_mtx = tf.math.confusion_matrix(labels, predictions)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, xticklabels=['Clear', 'SlantingHeavyRain', 'VerticalHeavyRain',
                                        'SlantingMediumRain', 'VerticalMediumRain', 'SlantingLowRain',
                                        'VerticalLowRain'],
            yticklabels=['Clear', 'SlantingHeavyRain', 'VerticalHeavyRain',
                        'SlantingMediumRain', 'VerticalMediumRain', 'SlantingLowRain',
                        'VerticalLowRain'],
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

In [ ]:
# Confution Matrix and Classification Report

print('Confusion Matrix')
print(confusion_matrix(labels, predictions))
print('Classification Report')
target_names = ['Clear', 'SlantingHeavyRain', 'VerticalHeavyRain',
                'SlantingMediumRain', 'VerticalMediumRain', 'SlantingLowRain',
                'VerticalLowRain']
print(classification_report(labels, predictions, target_names=target_names))

In [ ]:
filepath = "SqueezeNet"
tf.keras.models.save_model(model, filepath, include_optimizer=True)
keras_file = filepath
print('Saved baseline model to:', keras_file)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('TFliteSqueezeNet/model.tflite', 'wb') as f:
    f.write(tflite_model)